In [ ]:
import tiktoken
import pandas as pd

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
train_df =pd.read_csv('data/train_24892.csv')
test_df = pd.read_csv('data/test_13786.csv')
print(train_df.shape, test_df.shape)

In [ ]:
total_num = 0
for i in range(len(train_df['code'])):
    num = num_tokens_from_string(train_df['code'][i], "cl100k_base")
    if num >8192:
        print(i, num)
    total_num += num
print('total:' + str(total_num))
    

In [ ]:
total_num = 0
for i in range(len(test_df['code'])):
    num = num_tokens_from_string(test_df['code'][i], "cl100k_base")
    if num >8192:
        print(i, num)
    total_num += num
print('total:' + str(total_num))

In [ ]:
# !pip install openai==0.28.0

In [ ]:
import os
import random
import openai
from scipy.spatial import distance
from sklearn.cluster import KMeans
import tiktoken
import pandas as pd
import time

openai.api_key = "******"

In [ ]:
def get_embedding(text_to_embed):
    encoding = tiktoken.get_encoding("cl100k_base")
    encoded_string = encoding.encode(text_to_embed)
    num_tokens = len(encoded_string)
    if num_tokens > 8192:
	    print('Exceed max length,'+str(num_tokens))
	    string = encoding.decode(encoded_string[:8192])
    else:
	    string = text_to_embed
    
    for delay_secs in (2**x for x in range(0, 6)):
        try:
            response = openai.Embedding.create(
                model= "text-embedding-3-large",
                input=[string]
            )
            break
        except openai.OpenAIError as e:
            randomness_collision_avoidance = random.randint(0, 1000) / 1000.0
            sleep_dur = delay_secs + randomness_collision_avoidance
            print(f"Error: {e}. Retrying in {round(sleep_dur, 2)} seconds.")
            time.sleep(sleep_dur)
            continue

    # Extract the AI output embedding as a list of floats
    embedding = response["data"][0]["embedding"]
    
    return embedding

In [ ]:
train_df =pd.read_csv('data/train_24892.csv', index_col=0)
test_df = pd.read_csv('data/test_13786.csv', index_col=0)


print("Data train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

In [ ]:
# min_df = test_df[:10]

# min_df["embedding"] = min_df["code"].astype(str).apply(get_embedding)

# # Make the index start from 0
# min_df.reset_index(drop=True)

# min_df

In [ ]:
test_df["embedding"] = test_df["code"].astype(str).apply(get_embedding)

# Make the index start from 0
test_df.reset_index(drop=True)

test_df.head(10)

In [ ]:
test_df.to_csv('out/gpt4_test.csv', index=False)

In [ ]:
train_df["embedding"] = train_df["code"].astype(str).apply(get_embedding)

# Make the index start from 0
train_df.reset_index(drop=True)

train_df.head(10)

In [ ]:
train_df.to_csv('out/gpt4_train.csv', index=False)

## Evaluate the embeddings

In [1]:
import pandas as pd
import torch
import numpy as np
from ast import literal_eval

train_df = pd.read_csv('out/gpt4_train.csv')
test_df = pd.read_csv('out/gpt4_test.csv')

In [2]:
train_df["embedding"]=train_df["embedding"].apply(literal_eval)
train_df.embedding.map(lambda x: x[0])
train_list=list(train_df.embedding.values)
train_array=np.array(train_list)
train_tensor=torch.tensor(train_array, dtype=torch.float64)
torch.save(train_tensor, 'out/gpt4_train.pt')

In [8]:
test_df["embedding"]=test_df["embedding"].apply(literal_eval)
test_df.embedding.map(lambda x: x[0])
test_list=list(test_df.embedding.values)
test_array=np.array(test_list)
test_tensor=torch.tensor(test_array, dtype=torch.float64)
torch.save(test_tensor, 'out/gpt4_test.pt')

In [4]:
import sys
import os
from pathlib import Path

sys.path.append(sys.path[0] + '/src')

from prediction import analyze_data
import utils
import pandas as pd

In [5]:
device = utils.getting_device()

train_df =pd.read_csv('data/train_24892.csv')
test_df = pd.read_csv('data/test_13786.csv')
print(train_df.shape, test_df.shape)

model = 'gpt4'
train_file = 'out/'+model+'_train.pt'
test_file = 'out/'+model+'_test.pt'
bert_dic_lang, bert_dic_task = analyze_data(model, train_df, test_df, device, train_path = train_file, test_path = test_file, mode = 'two_label')

device: cuda
(24892, 4) (13786, 4)
Time elapsed: 2.36 seconds, Data predicted: 1
Time elapsed: 1059.84 seconds, Data predicted: 5001
Time elapsed: 1063.89 seconds, Data predicted: 10001
Accuracy of Programming Language prediction: 53.51%
Accuracy of Programming Task prediction: 76.21%


In [7]:
device = utils.getting_device()

train_df =pd.read_csv('out/gpt4_train.csv')
test_df = pd.read_csv('out/gpt4_test.csv')
print(train_df.shape, test_df.shape)

model = 'gpt4'
train_file = 'out/'+model+'_train.pt'
test_file = 'out/'+model+'_test.pt'
bert_dic_lang, bert_dic_task = analyze_data(model, train_df, test_df, device, train_path = train_file, test_path = test_file, mode = 'two_label')

device: cuda
(24892, 4) (13786, 4)
Time elapsed: 2.09 seconds, Data predicted: 1
Time elapsed: 1075.82 seconds, Data predicted: 5001
Time elapsed: 1076.62 seconds, Data predicted: 10001
Accuracy of Programming Language prediction: 53.51%
Accuracy of Programming Task prediction: 76.21%
